# Bag of Words and Vector Representations - Assignment 3:

<a target="_blank" href="https://colab.research.google.com/github/sham-nlp/2026nlp-3-bow-and-vector-representations/blob/main/03_bag_of_words_and_vector_representations_assignment_student.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

**Name:** `Your Name Here`

**Date:** `Insert Date`

---


This notebook is designed as hands-on practice for:
- Bag of Words (BoW)
- Embedding layers & sparsity in PyTorch
- Semantic similarity (cosine) + simple visualization (PCA)
- Challenge [Optional]: Training a small **Word2Vec CBOW** model on a toy dataset


## Instructions
1. Run the setup cell as it is. You do not need to change anything there.
2. Complete the code sections marked with `# YOUR CODE HERE` and `# YOUR CODE IN _`.
3. Run cells top-to-bottom in order.
4. **Submission**: Submit this notebook with all cells executed and outputs visible.

## Part 1: Setup (Toy Dataset + Helpers)

In [ ]:
# TODO: Run this cell first.

import re
import math
import random
from collections import Counter, defaultdict

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

# For plots (Ex03)
import matplotlib.pyplot as plt

# Reproducibility
SEED = 42 # This value ensures that if you run the notebook again, you will get the same results :)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

toy_sentences = [
    "Ramadan Kareem to everyone",
    "The month of Ramadan is here",
    "Ramadan is a blessed month",
    "I love cats",
    "I hate cats",
    "Cats are cute",
    "I love coffee",
    "Coffee is amazing",
    "I enjoy machine learning",
    "Deep learning is part of machine learning",
    "I enjoy natural language processing",
    "Natural language processing uses vectors",
]

def simple_tokenize(text: str):
    """Lowercase + keep words only (simple tokenizer for teaching)."""
    # We will study Tokenization in depth in a future session. For now, we get the tokens by splitting the words inside the sentence.
    text = text.lower()
    # keep alphabetic tokens
    return re.findall(r"[a-z]+", text)

tokenized = [simple_tokenize(s) for s in toy_sentences]

print("Example sentence:", toy_sentences[0])
print("Tokens:", tokenized[0])
print("Number of sentences:", len(toy_sentences))

Example sentence: Ramadan Kareem to everyone
Tokens: ['ramadan', 'kareem', 'to', 'everyone']
Number of sentences: 12


---

## Part 2: Bag of Words

Goal: transform a sentence into a Bag-of-Words (count vector).

You'll:
1. Build a vocabulary from the toy dataset.
2. Convert one sentence into a BoW vector.
3. (Optional) Create BoW for *all* sentences.

**Reminder:** BoW ignores word order.

<details>
<summary>Hint 1!</summary>

Flatten `tokenized` into a single list of all tokens, then use `set()` for unique words.
</details>

<details>
<summary>Hint 2!</summary>

```python
all_tokens = [t for sent in tokenized for t in sent]
vocab = list(set(all_tokens))
word2idx = {w: i for i, w in enumerate(vocab)}
```
</details>

In [ ]:
# TODO 1: Build a vocabulary from the dataset.
# For this exercise, you should include all unique tokens.
#
# Output:
# - vocab: list of tokens
# - word2idx: dict mapping token -> index. The idea is to assign an id (number) to each word. This index will be used as the order in the vocab list


all_tokens = # YOUR CODE HERE # You can use the tokens from the tokenized list
vocab = # YOUR CODE HERE
word2idx = # YOUR CODE HERE

print("Vocab size:", len(vocab))
print("First 15 vocab items:", vocab[:15])

<details>
<summary>Hint 1!</summary>

Create `torch.zeros(len(word2idx))`, then loop over tokens and increment at `word2idx[token]`.
</details>

<details>
<summary>Hint 2!</summary>

```python
bow = torch.zeros(len(word2idx))
for t in tokens:
    bow[word2idx[t]] += 1
return bow
```
</details>

In [ ]:
# TODO 2: Write a function that converts a tokenized sentence into a BoW vector (counts).
#
# Output:
# - bow: torch tensor of shape [|V|] with counts

def sentence_to_bow(tokens, word2idx):
    # YOUR CODE HERE
    # TODO: create a zero vector, then increment indices for each token
    return bow

# Test on a sentence:
test_sentence = "Ramadan is a blessed month"
test_tokens = simple_tokenize(test_sentence)
print("Test tokens:", test_tokens)

bow = sentence_to_bow(test_tokens, word2idx)
print(bow)
print("BoW shape:", bow.shape)
print("Non-zero entries:", int((bow > 0).sum()))

<details>
<summary>Hint 1!</summary>
Use sentence_to_bow to iterate over all tokens in `tokenized`
See https://docs.pytorch.org/docs/stable/generated/torch.stack.html
</details>

<details>
<summary>Hint 2!</summary>

`torch.stack([sentence_to_bow(t, word2idx) for t in tokenized])`
</details>

In [ ]:
# TODO 3 (Optional): Build a BoW matrix for all sentences.
# Output:
# - bow_matrix: tensor of shape [num_sentences, |V|]

bow_matrix = # YOUR CODE HERE

print("BoW matrix shape:", bow_matrix.shape)

---

## Part 3:  Embedding Layer, Vocab, and Sparse Gradients

Goal: Understand how to:
- define a vocabulary and token IDs
- use `nn.Embedding`
- compare gradients **with and without** `sparse=True`
- observe how many embedding rows receive non-zero gradients

**Key idea:** Only tokens that appear in the batch get gradients.  
With `sparse=True`, gradients are stored sparsely (only updated rows), which can save memory for huge vocabularies.

> Note: Not all optimizers support sparse gradients (e.g., use `SparseAdam` or `Adagrad`).

In [ ]:
# We'll reuse vocab/word2idx from Part 01

vocab = ['a', 'amazing', 'are', 'blessed', 'cats', 'coffee', 'cute', 'deep', 'enjoy', 'everyone', 'hate', 'here', 'i', 'is', 'kareem',
 'language', 'learning', 'love', 'machine', 'month', 'natural', 'of', 'part', 'processing', 'ramadan', 'the', 'to', 'uses', 'vectors']

word2idx= {'a': 0,
 'amazing': 1,
 'are': 2,
 'blessed': 3,
 'cats': 4,
 'coffee': 5,
 'cute': 6,
 'deep': 7,
 'enjoy': 8,
 'everyone': 9,
 'hate': 10,
 'here': 11,
 'i': 12,
 'is': 13,
 'kareem': 14,
 'language': 15,
 'learning': 16,
 'love': 17,
 'machine': 18,
 'month': 19,
 'natural': 20,
 'of': 21,
 'part': 22,
 'processing': 23,
 'ramadan': 24,
 'the': 25,
 'to': 26,
 'uses': 27,
 'vectors': 28}


<details>
<summary>Hint 1!</summary>

For each sentence, convert each token to its ID using `word2idx`.
</details>

<details>
<summary>Hint 2!</summary>

`[[word2idx[t] for t in sent] for sent in batch_tokens]`
</details>

In [ ]:
# Let's pick a mini-batch of sentences and turn them into token IDs.
batch_texts = [
    "ramadan is a blessed month",
    "the month of ramadan is here",
    "natural language processing uses vectors",
]

batch_tokens = [simple_tokenize(s) for s in batch_texts]
print("Batch tokens:", batch_tokens)

# TODO 2: Convert tokens -> IDs.

batch_ids = # YOUR CODE HERE # Hint: You should use the idx2word you created earlier!

print("Batch ids:", batch_ids)

<details>
<summary>Hint!</summary>

`num_embeddings=len(vocab)`, `embedding_dim=EMB_DIM`

Docs: https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html
</details>

In [ ]:
# Helper: mean pooling to get a sentence vector from word vectors.
def mean_pool(embeddings: torch.Tensor):
    """embeddings: [seq_len, dim] -> [dim]"""
    return embeddings.mean(dim=0)

# We'll compare embedding gradients with sparse=False vs sparse=True
EMB_DIM = 16

def run_embedding_grad_demo(sparse: bool):
    emb = nn.Embedding(num_embeddings=# YOUR CODE HERE,
                       embedding_dim=# YOUR CODE HERE,
                       sparse=sparse)

    # Create a single training step: predict a fake scalar target from sentence embeddings.
    # We'll create sentence vectors by mean pooling and then score them with a linear layer.
    scorer = nn.Linear(EMB_DIM, 1)

    # Build a batch tensor: pad to same length for convenience
    max_len = max(len(x) for x in batch_ids)
    pad_id = word2idx[vocab[0]]  # any id (we'll mask it out); for real work use a PAD token.

    batch = []
    mask = []
    for seq in batch_ids:
        padded = seq + [pad_id] * (max_len - len(seq))
        m = [1]*len(seq) + [0]*(max_len - len(seq))
        batch.append(padded)
        mask.append(m)

    batch = torch.tensor(batch)         # [B, L]
    mask = torch.tensor(mask).float()   # [B, L]

    # Forward
    word_vecs = emb(batch)              # [B, L, D]
    # mask out padded positions:
    word_vecs = word_vecs * mask.unsqueeze(-1)
    sent_vecs = word_vecs.sum(dim=1) / (mask.sum(dim=1, keepdim=True) + 1e-9)  # [B, D]
    scores = scorer(sent_vecs).squeeze(-1)  # [B]

    # Fake regression target
    target = torch.tensor([1.0, 0.0, 1.0])

    loss = F.mse_loss(scores, target)
    loss.backward()

    grad = emb.weight.grad
    return loss.item(), grad

# TODO 3: Run both variants and inspect the gradient structure.

loss_dense, grad_dense = run_embedding_grad_demo(sparse=False)
loss_sparse, grad_sparse = run_embedding_grad_demo(sparse=True)

print("Loss (dense): ", loss_dense)
print("Loss (sparse):", loss_sparse)

print("\nDense grad type:", type(grad_dense), "shape:", grad_dense.shape)
print("Sparse grad type:", type(grad_sparse), "is_sparse:", getattr(grad_sparse, "is_sparse", False))


<details>
<summary>Hint 1 (TODO 4)!</summary>

`(row_sums > 0).sum().item()`
</details>

<details>
<summary>Hint 2 (TODO 5)!</summary>

`grad_sparse._indices()[0].tolist()`
</details>

<details>
<summary>Hint 3 (TODO 6)!</summary>

`set(updated_rows_sparse) == set(torch.where(row_sums > 0)[0].tolist())`
</details>

In [ ]:
# TODO 4: Count how many rows have non-zero gradients in the dense case.

row_sums = grad_dense.abs().sum(dim=1)
nonzero_rows_dense = # YOUR CODE HERE

print("Non-zero gradient rows (dense):", nonzero_rows_dense, "out of", len(vocab))

# TODO 5: For sparse gradients, inspect which rows are updated.
# HINT: grad_sparse._indices() gives the row indices

updated_rows_sparse = # YOUR CODE HERE

print("Updated rows (sparse):", updated_rows_sparse)
print("Number of updated rows (sparse):", len(updated_rows_sparse))

# TODO 6: Compare the sets of updated rows (dense vs sparse) - should match.

# YOUR CODE HERE

---

## Part 4: Semantic Similarity + Visualization (Cosine + PCA)

Goal:
1. Create **sentence vectors** (simple approach: mean pooling of word embeddings).
2. Use **cosine similarity** to find the closest sentences to a query sentence (like a mini recommender).
3. Visualize sentence vectors after **dimensionality reduction** (PCA to 2D).

For this exercise, we will use a pretrained light glove model. Follow the instructions and run the cells to download the model

**Important Note** If you face any problems during the model download (due to connection issue), you can use a raw Embedding layer from Pytorch. However, keep in mind that the results of the similarity can be very weak with this layer.

### Using Glove Model

In [ ]:
!pip install gensim

In [ ]:
import gensim.downloader as api
model = api.load("glove-wiki-gigaword-100")

<details>
<summary>Hint!</summary>

`model[word]` returns the embedding vector. Filter tokens that exist in the model:
`[model[t] for t in tokens if t in model]`
</details>

In [ ]:
import numpy as np

# TODO 1: Create an embedding layer and compute a sentence embedding for each sentence in toy_sentences.

def sentence_embedding(tokens):
    vecs = # YOUR CODE HERE
    return np.mean(vecs, axis=0)

sentence_vecs = np.stack([
    sentence_embedding(simple_tokenize(s))
    for s in toy_sentences
])

<details>
<summary>Hint (TODO 2)!</summary>

`cosine_similarity(sentence_vecs[query_idx:query_idx+1], sentence_vecs)[0]`

Docs: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html
</details>

<details>
<summary>Hint (TODO 3)!</summary>

`np.argsort(sims)[-k:][::-1]`
</details>

In [ ]:
# TODO 2: Cosine similarity based retrieval.
from sklearn.metrics.pairwise import cosine_similarity

query_idx = 0
sims = # YOUR CODE HERE

print("Query:", toy_sentences[query_idx])

# TODO 3: get top-k indices

top_idx = # YOUR CODE HERE
for i in top_idx:
    print(f"{sims[i]:.3f} | {toy_sentences[i]}")


<details>
<summary>Hint!</summary>

```python
pca = PCA(n_components=2)
X2 = pca.fit_transform(sentence_vecs)
```

Docs: https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html
</details>

In [ ]:
# TODO 4: PCA visualization in 2D.
# - Reduce sentence_vecs [N, D] to [N, 2]
# - Plot points and label them with their sentence index


from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# YOUR CODE HERE
pca = # YOUR CODE HERE
X2 = # YOUR CODE HERE

plt.scatter(X2[:,0], X2[:,1])
for i, (x,y) in enumerate(X2):
    plt.text(x+0.01, y+0.01, str(i))
plt.show()


for i, s in enumerate(toy_sentences):
    print(i, "->", s)

### Using Embedding Layer from Pytorch


<details>
<summary>Hint 1 (vecs)!</summary>

`emb(ids)` — pass the tensor through the embedding layer.
</details>

<details>
<summary>Hint 2 (sentence_vecs)!</summary>

`torch.stack([sentence_embedding(t) for t in tokenized])`
</details>

In [ ]:
# We'll build sentence vectors using an embedding layer.
# TODO 1: Create an embedding layer and compute a sentence embedding for each sentence in toy_sentences.

EMB_DIM = 32
emb = nn.Embedding(num_embeddings=len(vocab), embedding_dim=EMB_DIM)

def sentence_to_ids(tokens):
    return [word2idx[t] for t in tokens]

def sentence_embedding(tokens):
    ids = torch.tensor(sentence_to_ids(tokens))
    vecs =  # YOUR CODE HERE      # [L, D]
    sent_vec = vecs.mean(dim=0)  # [D]
    return sent_vec

# Build a matrix: [N, D]
# TODO: compute sentence_vecs by stacking sentence_embedding(tokenized[i])
sentence_vecs = # YOUR CODE HERE

print("Sentence vectors shape:", sentence_vecs.shape)

<details>
<summary>Hint 1 (sims)!</summary>

`F.cosine_similarity(query.unsqueeze(0), sentence_vecs, dim=1)`

Docs: https://pytorch.org/docs/stable/generated/torch.nn.functional.cosine_similarity.html
</details>

<details>
<summary>Hint 2 (topk)!</summary>

`torch.topk(sims, k=k)`
</details>

In [ ]:
# TODO 2: Cosine similarity based retrieval.
# Given a query sentence index, return the top-k most similar other sentences.

def topk_similar(query_idx: int, k: int = 3):
    query = sentence_vecs[query_idx]              # [D]
    # TODO: compute cosine similarity between query and all sentence_vecs -> [N]
    # HINT: F.cosine_similarity function can be used here

    sims = # YOUR CODE HERE

    # Exclude the query itself
    sims = sims.clone()
    sims[query_idx] = -1e9

    # TODO: get top-k indices
    vals, idxs = # YOUR CODE HERE

    return idxs, vals

query_idx = 0
topk, scores = topk_similar(query_idx, k=4)
print("Query:", toy_sentences[query_idx])
for i, s in zip(topk.tolist(), scores.tolist()):
    print(f"  sim={s:.3f} | {toy_sentences[i]}")

<details>
<summary>Hint!</summary>

```python
pca = PCA(n_components=2)
X2 = pca.fit_transform(sentence_vecs.detach().numpy())
```
</details>

In [ ]:
# TODO 3: PCA visualization in 2D.
# - Reduce sentence_vecs [N, D] to [N, 2]
# - Plot points and label them with their sentence index

from sklearn.decomposition import PCA

# YOUR CODE HERE
pca = # YOUR CODE HERE
X2 = # YOUR CODE HERE

plt.figure(figsize=(7, 6))
plt.scatter(X2[:, 0], X2[:, 1])

for i, (x, y) in enumerate(X2):
    plt.text(x + 0.01, y + 0.01, str(i), fontsize=9)

plt.title("Sentence vectors (PCA to 2D) — indices correspond to toy_sentences")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.show()

print("Index -> sentence")
for i, s in enumerate(toy_sentences):
    print(i, "->", s)

---

## Part 5 Challenge! [Optional]: Train a Word2Vec Model (CBOW) on the Toy Dataset

CBOW objective: **given context words, predict the center word**.

We'll implement:
- dataset creation: (context_ids, target_id)
- a CBOW model with `nn.Embedding` + mean pooling + linear classifier
- a training loop with cross entropy loss

Because the dataset is tiny, don't expect perfect semantics, focus on the mechanics.

In [ ]:
# You do not need to change this cell. And it is okay if you do not understand token-related codes for now. We'll cover them in later sessions inshaa Allah

# Build CBOW training examples.
WINDOW = 2 # We'll use a window size of 2: i.e. two words before and two words after the center word. See the following example:
# For tokens: [w0, w1, w2, w3, w4]
# Center= w2, context=[w0,w1,w3,w4]

def build_cbow_examples(tokenized_sentences, window=2):
    examples = []
    for sent in tokenized_sentences:
        ids = [word2idx[t] for t in sent]
        for center in range(len(ids)):
            left = max(0, center - window)
            right = min(len(ids), center + window + 1)
            context = [ids[i] for i in range(left, right) if i != center]
            target = ids[center]
            # Skip if context is empty (very short sentences)
            if len(context) == 0:
                continue
            examples.append((context, target))
    return examples

examples = build_cbow_examples(tokenized, window=WINDOW)
print("Number of CBOW examples:", len(examples))
print("First 5 examples (context_ids, target_id):", examples[:5])

# Make a simple collate function that pads contexts to same length in a batch.
# Output:
# - contexts: LongTensor [B, L]
# - mask: FloatTensor [B, L] 1 for real tokens, 0 for padding
# - targets: LongTensor [B]

PAD_ID = word2idx[vocab[0]]  # (for teaching only; normally add an explicit <PAD> token)

def collate_batch(batch):
    max_len = max(len(ctx) for ctx, _ in batch)
    contexts, masks, targets = [], [], []
    for ctx, tgt in batch:
        pad_len = max_len - len(ctx)
        contexts.append(ctx + [PAD_ID]*pad_len)
        masks.append([1]*len(ctx) + [0]*pad_len)
        targets.append(tgt)
    return torch.tensor(contexts), torch.tensor(masks).float(), torch.tensor(targets)

# Quick sanity check:
contexts, masks, targets = collate_batch(examples[:4])
print("contexts:", contexts)
print("masks:", masks)
print("targets:", targets)

<details>
<summary>Hint!</summary>

`{idx: word for word, idx in word2idx.items()}`
</details>

In [ ]:
# TODO 1: Create an idx2word mapping.
# This should map each number to a word, based on word2idx
idx2word = # YOUR CODE HERE

<details>
<summary>Hint 1 (forward)!</summary>

```python
embeddings = self.emb(context_ids)
masked = embeddings * mask.unsqueeze(-1)
pooled = masked.sum(dim=1) / (mask.sum(dim=1, keepdim=True) + 1e-9)
return self.linear(pooled)
```
</details>

<details>
<summary>Hint 2 (optimizer/criterion)!</summary>

`optimizer = torch.optim.Adam(model.parameters(), lr=0.01)`

`criterion = nn.CrossEntropyLoss()`
</details>

In [ ]:
# TODO 2: Implement the CBOW model.

class CBOW(nn.Module):
    def __init__(self, vocab_size: int, emb_dim: int):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim)
        self.linear = nn.Linear(emb_dim, vocab_size)

    def forward(self, context_ids: torch.Tensor, mask: torch.Tensor):
        # context_ids: [B, L]
        # mask: [B, L] (1 for real, 0 for padding)

        # TODO: look up embeddings -> [B, L, D]
        # YOUR CODE HERE
        # TODO: mask out padding, then mean-pool over L -> [B, D]
        # YOUR CODE HERE
        # TODO: logits = linear(pooled) -> [B, V]
        # YOUR CODE HERE
        return # YOUR CODE HERE


VOCAB_SIZE = len(vocab)
EMB_DIM = 32
model = CBOW(VOCAB_SIZE, EMB_DIM)

# TODO 3: Choose an optimizer and a loss.

optimizer = # YOUR CODE HERE
criterion = # YOUR CODE HERE

print(model)

<details>
<summary>Hint!</summary>

```python
optimizer.zero_grad()
logits = model(contexts, masks)
loss = criterion(logits, targets)
loss.backward()
optimizer.step()
```
</details>

In [ ]:
# TODO 4: Training loop
# Train for a few epochs and watch loss go down.

def iterate_minibatches(data, batch_size=16, shuffle=True):
    idxs = list(range(len(data)))
    if shuffle:
        random.shuffle(idxs)
    for start in range(0, len(data), batch_size):
        batch = [data[i] for i in idxs[start:start+batch_size]]
        yield batch

EPOCHS = 30
BATCH_SIZE = 32

for epoch in range(1, EPOCHS+1):
    total_loss = 0.0
    model.train()

    for batch in iterate_minibatches(examples, batch_size=BATCH_SIZE, shuffle=True):
        contexts, masks, targets = collate_batch(batch)

        # TODO: reset the gradients of the optimizer
        # YOUR CODE HERE
        # TODO: forward -> logits
        # YOUR CODE HERE
        # TODO: compute loss
        # YOUR CODE HERE
        # TODO: backward + step
        # YOUR CODE HERE

        total_loss += loss.item() * len(batch)

    avg_loss = total_loss / len(examples)
    if epoch % 5 == 0 or epoch == 1:
        print(f"Epoch {epoch:02d} | avg_loss={avg_loss:.4f}")

<details>
<summary>Hint!</summary>

`F.cosine_similarity(W, q.unsqueeze(0), dim=1)`
</details>

In [ ]:
# TODO 5: Inspect learned word neighbors (cosine similarity in embedding space).
# We'll compute cosine similarity between a query word embedding and all other words.

def most_similar_words(query_word: str, topk: int = 5):
    query_id = word2idx[query_word]
    W = model.emb.weight.detach()  # [V, D]
    q = W[query_id]                # [D]

    # TODO: cosine similarity between W and q -> [V]
    sims = # YOUR CODE HERE

    # Exclude itself
    sims[query_id] = -1e9
    vals, idxs = torch.topk(sims, k=topk)
    return [(idx2word[i.item()], vals[j].item()) for j, i in enumerate(idxs)]

# Try a few query words (if they exist in vocab)
for w in ["ramadan", "month", "cats", "coffee", "learning"]:
    if w in word2idx:
        print("\nQuery:", w)
        print(most_similar_words(w, topk=5))